In [1]:
import numpy as np
import time
import copy
import random
from time import process_time
'''initialization setting'''
num_job=40 # number of jobs
population_size=80
crossover_rate=0.8
mutation_rate=0.1
num_iteration=10000
replikasyon=10
'''processing time'''
p=[26,24,79,46,32,35,73,74,14,67,86,46,78,40,29,94,64,27,90,55,35,52,36,69,85,95,14,78,37,86,44,28,39,12,30,68,70,9,49,50]
'''weight'''
w=[1,10,9,10,10,4,3,2,10,3,7,3,1,3,10,4,7,7,4,7,5,3,5,4,9,5,2,8,10,4,7,4,9,5,7,7,5,10,1,3]
'''due date'''
d=[1588,1620,1731,1773,1694,1487,1566,1844,1727,1636,1599,1539,1855,1645,1709,1660,1582,1836,1484,1559,1772,1510,1512,1795,1522,1509,1598,1658,1826,1628,1650,1833,1627,1528,1541,1497,1481,1446,1579,1814]

In [2]:
def generateInitialPopulation(population_size):
    population_list=[]
    for i in range(population_size):
        #np.random.seed(i)
        random_num=list(np.random.permutation(num_job)) # generate a random permutation of 0 to num_job-1
        population_list.append(random_num) # add to the population_list
    return(population_list)

In [3]:
def fitnessValue(solution):
    ptime=0
    tardiness=0
    for j in range(num_job):
        ptime=ptime+p[solution[j]]
        tardiness=tardiness+w[solution[j]]*max(ptime-d[solution[j]],0)
    return (tardiness)

In [4]:
def binary_tournament(population):
    parent = []
    candidates = random.sample(population,2)
    tardiness1 = fitnessValue(candidates[0])
    tardiness2 = fitnessValue(candidates[1])
    if tardiness1 < tardiness2:
        parent = candidates[0]
    else:
        parent = candidates[1]
    return parent

In [5]:
'''MATING POOL'''
def select_parent(population):
    parent_pairs = []
    # randomly choose how many parent pairs will be selected
    parent_pair_count = random.randint(2, int(len(population)/2)) #number of pairs (2,populationSize/2)-->random
    for k in range(parent_pair_count):
        parent1 = binary_tournament(population)
        parent2 = binary_tournament(population)
        if parent1 != parent2 and (parent1, parent2) not in parent_pairs:
            parent_pairs.append((parent1, parent2))
    return parent_pairs

In [6]:
def crossover(parents):      #2-point crossover
    parent1 = parents[0]
    parent2 = parents[1]
    length_of_parent = len(parent1)
    points=random.sample(range(1, length_of_parent), 2) # 2 random points , range (1,length_of_parent)
    if points[0]<points[1]:
        first_point=points[0]
        second_point=points[1]
    else:
        first_point=points[1]
        second_point=points[0]
    intersect1 = parent1[first_point:second_point]
    intersect2 = parent2[first_point:second_point]

    child1 = []
    child2 = []
    index1 = 0
    for pos2 in range(len(parent2)):
        if first_point <= index1 < second_point:
            child1.extend(intersect1)
            index1 = second_point
        if parent2[pos2] not in intersect1:
                child1.append(parent2[pos2])
                index1 += 1
    index2 = 0
    for pos2 in range(len(parent1)):
        if first_point <= index2 < second_point:
            child2.extend(intersect2)
            index2 = second_point
        if parent1[pos2] not in intersect2:
                child2.append(parent1[pos2])
                index2 += 1

    return child1, child2

In [7]:
def mutation(solution):
    # copy the solution
    mutated_solution = list(solution)
    solution_length = len(solution)
    # pick 2 positions to swap randomly
    swap_positions = list(np.random.permutation(np.arange(solution_length))[:2])
    first_job = solution[swap_positions[0]]
    second_job = solution[swap_positions[1]]
    mutated_solution[swap_positions[0]] = second_job
    mutated_solution[swap_positions[1]] = first_job
    return mutated_solution

In [8]:
def update_population(population, children):
    costed_population = []
    for individual in population:
        ind_tardiness = (fitnessValue(individual), individual)
        costed_population.append(ind_tardiness)
    costed_population.sort(key=lambda x: x[0], reverse=True)  #descending order

    costed_children = []
    for individual in children:
        ind_tardiness = (fitnessValue(individual), individual)
        costed_children.append(ind_tardiness)
    costed_children.sort(key=lambda x: x[0])
    for child in costed_children:
        if child not in population:
            population.append(individual)
            population.remove(costed_population[0][1])
            break
    return(population)

In [9]:
"""MAIN CODE"""
for m in range (replikasyon):
    # Initialize population
    population = generateInitialPopulation(population_size)

    # Start time for CPU calculation
    start_time = process_time()

    for evaluation in range(num_iteration):
        # Select parents
        parent_list = select_parent(population)
        childs = []

        # Apply crossover to generate children
        for parents in parent_list:
            r = np.random.rand()
            if r < crossover_rate:
                childs.extend(crossover(parents))
            else:
                childs.append(parents[0])
                childs.append(parents[1])

        # Apply mutation operation to change the order of the n-jobs
        mutated_childs = []
        for child in childs:
            r = np.random.rand()
            if r < mutation_rate:
                mutated_child = mutation(child)
                mutated_childs.append(mutated_child)

        childs.extend(mutated_childs)
        if len(childs) > 0:
            update_population(population, childs)
        

    # End time for CPU calculation        
    end_time = process_time()

    costed_population = []
    for individual in population:
        ind_tardiness = (fitnessValue(individual), individual)
        costed_population.append(ind_tardiness)
    costed_population.sort(key=lambda x: x[0])

    avgObjective = sum(t[0] for t in costed_population) / len(costed_population)
    bestObjective = costed_population[0][0]

    print("Solution(sequence of jobs processed by the machines):", costed_population[0][1])
    print("Objective Value of the Best Chromosome:", str(bestObjective))
    print("Average Objective Value of All Chromosomes:", "%.2f" %avgObjective)
    print("CPU Time (s)", "%.2f" %(end_time - start_time))
    #print(costed_population)

    '''PRINT TO TXT FILE'''
    with open("replikasyon_{0}.txt".format(m), 'w') as f:
        print("Solution(sequence of jobs processed by the machines):", costed_population[0][1], file=f)
        print("Objective Value of the Best Chromosome:", str(bestObjective), file=f)
        print("Average Objective Value of All Chromosomes:", "%.2f" %avgObjective, file=f)
        print("CPU Time (s)", "%.2f" %(end_time - start_time), file=f) 
        print(costed_population, file=f)

Solution(sequence of jobs processed by the machines): [32, 5, 13, 29, 25, 20, 34, 10, 4, 17, 33, 21, 31, 1, 14, 24, 37, 3, 36, 27, 9, 18, 30, 0, 26, 22, 19, 16, 8, 11, 35, 6, 15, 2, 23, 28, 39, 7, 38, 12]
Objective Value of the Best Chromosome: 956
Average Objective Value of All Chromosomes: 956.10
CPU Time (s) 51.53
Solution(sequence of jobs processed by the machines): [35, 19, 11, 28, 36, 26, 30, 18, 38, 8, 0, 34, 16, 32, 17, 1, 33, 24, 27, 37, 22, 5, 25, 21, 29, 14, 10, 4, 13, 6, 3, 9, 15, 2, 20, 39, 31, 23, 7, 12]
Objective Value of the Best Chromosome: 1018
Average Objective Value of All Chromosomes: 1030.56
CPU Time (s) 54.97
Solution(sequence of jobs processed by the machines): [24, 14, 33, 25, 5, 30, 35, 21, 18, 8, 37, 3, 38, 16, 32, 20, 26, 11, 6, 15, 4, 9, 1, 10, 13, 19, 0, 34, 36, 22, 27, 29, 2, 23, 28, 17, 31, 39, 7, 12]
Objective Value of the Best Chromosome: 913
Average Objective Value of All Chromosomes: 976.70
CPU Time (s) 54.31
Solution(sequence of jobs processed by th